<a href="https://colab.research.google.com/github/TomPhanAnh/FPT-Data-Science/blob/DSP302x_Data-Analysis-with-Python/DSP302x_ASM1_A_VN__AnhptFX12081_funix_edu_vn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sqlite3 
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns",100)
pd.set_option("display.max_colwidth",-1)

In [ ]:
# Câu 1: Kết nối với database và lấy thông tin của tất cả các bảng.
conn = sqlite3.connect(r"C:/Users/ptama/Downloads/database.sqlite")
# Get information of all table
tables = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type = 'table'", conn)['name']

In [ ]:
tables

0    sqlite_sequence  
1    Player_Attributes
2    Player           
3    Match            
4    League           
5    Country          
6    Team             
7    Team_Attributes  
Name: name, dtype: object

In [ ]:
# Câu hỏi 2: Chọn dữ liệu trong bảng "Country"
C2 = pd.read_sql_query('SELECT * FROM Country',conn)

In [ ]:
df_countries.rename(columns = {'id':'country_id'}, inplace = True)

In [ ]:
# np.delete is for delete a first (0) value of tables 
# df.nunique Count number of distinct elements in specified axis.
for table in np.delete(tables.values,0):
    df_tmp = pd.read_sql_query('SELECT * FROM {}'.format(table), conn)
    print(80*('-'))
    print("Table Name: {}--- Rows: {} --- Columns: {} --- Unique values: ---{}".
         format(table,df_tmp.shape[0],df_tmp.shape[1], df_tmp.nunique().sum()))
    print('Null value -----------')
    navals = df_tmp.isna().sum()
    print('Total Null Values: {}'.format(navals.sum()))
    print("Duplicated Record: {}".format(df_tmp.duplicated().sum()))
    print(navals)

--------------------------------------------------------------------------------
Table Name: Player_Attributes--- Rows: 183978 --- Columns: 42 --- Unique values: ---209464
Null value -----------
Total Null Values: 47301
Duplicated Record: 0
id                     0   
player_fifa_api_id     0   
player_api_id          0   
date                   0   
overall_rating         836 
potential              836 
preferred_foot         836 
attacking_work_rate    3230
defensive_work_rate    836 
crossing               836 
finishing              836 
heading_accuracy       836 
short_passing          836 
volleys                2713
dribbling              836 
curve                  2713
free_kick_accuracy     836 
long_passing           836 
ball_control           836 
acceleration           836 
sprint_speed           836 
agility                2713
reactions              836 
balance                2713
shot_power             836 
jumping                2713
stamina                836 
str

In [ ]:
# Get the matches played in one season
matches_season = pd.read_sql_query('Select count(*) as match_played,season from Match group by season', conn)
matches_season

,match_played,season
0,3326,2008/2009
1,3230,2009/2010
2,3260,2010/2011
3,3220,2011/2012
4,3260,2012/2013
5,3032,2013/2014
6,3325,2014/2015
7,3326,2015/2016


In [ ]:
# check for null values
print("Null value {}".format(matches_season.isna().sum()))
print(100*("-"))
print("Duplicated value: {}".format(matches_season.duplicated().sum()))

Null value match_played    0
season          0
dtype: int64
----------------------------------------------------------------------------------------------------
Duplicated value: 0


In [ ]:
# Câu hỏi 3: Chọn dữ liệu trong bảng "League"
C3 = pd.read_sql_query('''
SELECT * FROM Team''', conn)

In [ ]:
#  Câu hỏi 4: Chọn dữ liệu League với thông tin quốc gia
C4 = pd.read_sql_query('''SELECT * FROM League l JOIN Country c ON l.country_id = c.id ''', conn)

In [ ]:
# Câu hỏi 5: Chọn dữ liệu từ bảng “Match”
C5 = pd.read_sql_query('Select * from Match',conn)

In [ ]:
# Câu hỏi 6: Chọn dữ liệu trận đấu bao gồm  thông tin giải đấu và quốc gia
C6 = pd.read_sql_query('''Select * from Country c 
                                            JOIN (League l JOIN Match m  on l.country_id = m.country_id) on c.id = m.country_id''' , conn)

In [ ]:
# Câu hỏi 7: Lấy số lượng trận đấu của mỗi giải đấu bao gồm cả tên giải đấu, sắp xếp theo thứ tự giảm dần số trận đấu
C7 = pd.read_sql_query('''
SELECT count(*) as matches, name , season 
FROM ( SELECT * FROM Match m JOIN League l ON m.country_id = l.country_id)  
GROUP BY name, season
ORDER BY matches DESC''', conn)


In [ ]:
# Câu hỏi 8: Lấy tổng số bàn thắng của đội nhà và đội khách trong mỗi giải đấu
C8 = pd.read_sql_query('''
SELECT name, season, sum(home_team_goal) as home_total_goal, 
        sum(away_team_goal) as away_total_home
FROM (
SELECT * 
FROM Match m JOIN League l ON m.country_id = l.country_id)
GROUP BY name, season
 ''', conn)

In [ ]:
# Câu hỏi 9: Chọn dữ liệu từ bảng “Team”
C9 = pd.read_sql_query('''
SELECT * FROM Team''', conn)

In [ ]:
# Câu hỏi 10: Top 20 đội với số bàn thắng sân nhà cao nhất
C10 = pd.read_sql_query('''SELECT team_short_name, sum(home_team_goal) as total_goal 
                                FROM Match, Team where Match.id = Team.id 
                                GROUP BY team_short_name
                                ORDER BY total_goal DESC
                                LIMIT 20''', conn)
# Câu hỏi 11: Top 20 đội với số bàn thắng sân khách cao nhất
C11 = pd.read_sql_query('''SELECT team_short_name, sum(away_team_goal) as total_goal
                                FROM Match m JOIN Team t
                                ON m.id = t.id
                                GROUP BY team_short_name
                                ORDER BY total_goal DESC
                                LIMIT 20''', conn)

In [ ]:
#Câu hỏi 12: Liệt kê tên các đội bóng và tổng số bàn thắng mỗi đội, sắp xếp theo số lượng giảm dần
C12 = pd.read_sql_query('''
SELECT team_long_name, SUM(goal) as total_goal
FROM 
(SELECT t.team_long_name, SUM(m.home_team_goal) AS goal
FROM Match m, Team t
WHERE m.home_team_api_id = t.team_api_id
GROUP BY t.team_long_name

UNION 

SELECT t.team_long_name, SUM(m.away_team_goal) AS goal
FROM Match m, Team t
WHERE m.away_team_api_id = t.team_api_id
GROUP BY t.team_long_name)

GROUP BY team_long_name 
ORDER BY SUM(goal) DESC
''', conn)


In [ ]:
# Câu hỏi 13: Liệt kê tên các đội bóng (long name) và tổng số trận đấu
C13 = pd.read_sql_query('''
SELECT 
    team_long_name,
    sum(match)
FROM 
(SELECT
    t.team_long_name, count(*) as match
FROM Match m, Team t
WHERE m.home_team_api_id = t.team_api_id
GROUP BY t.team_long_name

UNION 

SELECT t.team_long_name, count(*) AS match
FROM Match m, Team t
WHERE m.away_team_api_id = t.team_api_id
GROUP BY t.team_long_name)

GROUP BY team_long_name
ORDER BY SUM(match) DESC''',
                       conn)

In [ ]:
# Câu hỏi 14: Liệt kê số trận thắng, thua và hòa của mỗi đội.
C14 = pd.read_sql_query('''
SELECT team_long_name, result, count(result) as game FROM 
(SELECT team_long_name, home_team_goal, away_team_goal,

CASE 
    WHEN home_team_goal > away_team_goal THEN "Win"
    WHEN home_team_goal < away_team_goal THEN "Lose"
    ELSE "Tie"
END AS result
FROM (
SELECT * FROM Match m JOIN Team t 
    ON m.home_team_api_id = t.team_api_id))
GROUP BY team_long_name, result''', conn)

In [ ]:
# Câu hỏi 15: Liệt kê 10 đội có số trận thắng nhiều nhất
C15 = pd.read_sql_query('''
SELECT team_long_name, count(result) as total_match

FROM (
SELECT team_long_name, home_team_goal, away_team_goal,

CASE 
    WHEN home_team_goal > away_team_goal THEN "Win"
    WHEN home_team_goal < away_team_goal THEN "Lose"
    ELSE "Tie"
END AS result

FROM (
SELECT * FROM Match m JOIN Team t ON m.home_team_api_id = t.team_api_id))
GROUP BY team_long_name, result
HAVING result = "Win"
ORDER BY total_match DESC
LIMIT 10''', conn)

In [ ]:
# Câu hỏi 16: Liệt kê tỉ lệ phần trăm về số trận của mỗi giải trên tổng tất cả các trận
C16 = pd.read_sql_query('''
SELECT name, 100*total_match/(
SELECT count(*) FROM Match m 
        JOIN League l on m.league_id = l.id) 
AS percentage

FROM (
SELECT name, count(*) AS total_match
FROM Match m JOIN League l on m.league_id = l.id
GROUP BY name)
ORDER BY percentage DESC
''', conn)

In [ ]:
# Câu hỏi 17: Liệt kê tỉ lệ phần trăm về số bàn ghi được của từng giải so với tổng số bàn của tất cả các trận.
C17 = pd.read_sql_query('''
SELECT name, 100*total_goal/(
SELECT sum(home_team_goal + away_team_goal)
FROM Match m JOIN League l ON m.league_id = l.id) AS percentage

FROM (SELECT name, sum(home_team_goal + away_team_goal) AS total_goal
FROM Match m JOIN League l on m.league_id = l.id
GROUP BY name)
ORDER BY percentage DESC''', conn)

In [ ]:
#Câu hỏi 18: Liệt kê tổng số bàn ghi được của từng giải đấu theo từng mùa giải.
C18 = pd.read_sql_query('''
SELECT name, season, sum(home_team_goal + away_team_goal) as total_goal
FROM Match m
JOIN League l
ON m.league_id = l.id
GROUP BY name, season
ORDER BY name,season ASC
''', conn)

In [ ]:
#Câu hỏi 19: Liệt kê thông tin cầu thủ:
C19 = pd.read_sql_query('''
SELECT weight*0.453592 as weight_kg,
    weight/100 as height_m2,
    weight*0.453592/(height*height/10000) as BMI
    FROM Player''', conn)

In [ ]:
#Câu hỏi 20: Liệt kê cầu thủ lớn tuổi nhất
C20 = pd.read_sql_query('''
SELECT *, DATE() - birthday as Age 
FROM Player
ORDER BY Age DESC 
LIMIT 1''', conn)